In [376]:
import numpy as np

INSPIRATION FROM THE FOLLOWING BLOG 

https://medium.com/aimonks/traveling-salesman-problem-tsp-using-genetic-algorithm-fea640713758

In [377]:
from random import randrange, random
from random import shuffle

In [4]:
# set up country list

cities = [
    "Paris", "Berlin", "London", "Madrid", "Rome", "Amsterdam", "Lisbon", "Prague", "Vienna", "Stockholm",
]

len(cities)

10

In [5]:
def determine_coords(city_list, max_distance):
    country_coords = {}
    for city in city_list:
        while len(country_coords) < len(city_list):
            coords = (randrange(0, max_distance), randrange(0, max_distance))
            if coords not in country_coords.values():
                country_coords[city] = coords
                break

    return country_coords

coords = determine_coords(cities, 20)
print(len(coords))

coords

10


{'Paris': (8, 12),
 'Berlin': (5, 19),
 'London': (3, 11),
 'Madrid': (19, 14),
 'Rome': (16, 5),
 'Amsterdam': (2, 9),
 'Lisbon': (1, 9),
 'Prague': (1, 12),
 'Vienna': (4, 3),
 'Stockholm': (10, 19)}

In [6]:
# out of above countries, select N countries to construct initial population

def initial_countries(n_sol, n_countries, country_list):
    selected_cities = []  # list of n countries to permute
    while len(selected_cities) < n_sol:
        sublist_cities = []  # list of n countries to permute
        while len(sublist_cities) < n_countries:
            rand = randrange(0, len(country_list))
            selected_city = country_list[rand]
            if selected_city not in sublist_cities:
                sublist_cities.append(selected_city)
        if len(set(sublist_cities)) == n_countries:  # Ensure unique cities
            selected_cities.append(sublist_cities)

    return selected_cities

cities_selection = initial_countries(100, 10, cities)

In [7]:
def construct_pop_coords(solution_arr, coord_map):
    result_dict_list = []

    for subarray in solution_arr:
        subarray_dict = {city: coord_map.get(city, None) for city in subarray}
        result_dict_list.append(subarray_dict)

    return result_dict_list

solutions_coords = construct_pop_coords(cities_selection, coords)

solutions_coords[0]

{'Berlin': (5, 19),
 'Vienna': (4, 3),
 'Prague': (1, 12),
 'Lisbon': (1, 9),
 'Madrid': (19, 14),
 'Stockholm': (10, 19),
 'Paris': (8, 12),
 'Rome': (16, 5),
 'London': (3, 11),
 'Amsterdam': (2, 9)}

In [8]:
from math import dist

In [9]:
def calc_total_dist(sol_coords):
    max_range = len(sol_coords[0]) - 1
    total_dist_per_sol = []
    for sol_coord in sol_coords:
        solution_total_dist = 0
        for i in range(max_range):
            city_1_coords = list(sol_coord.values())[i]
            city_2_coords = list(sol_coord.values())[i + 1]
            
            distance = dist(city_1_coords, city_2_coords)
            solution_total_dist += distance 
            
        total_dist_per_sol.append(solution_total_dist)
    return total_dist_per_sol
        
total_distances_per_sol = calc_total_dist(solutions_coords)

In [10]:
def assign_weights(distance_sum_list):
    score_list = []
    for distance_sum in distance_sum_list:
        score_list.append(1 / (1 + distance_sum))

    return score_list

weights = assign_weights(total_distances_per_sol)

len(weights)

100

In [11]:
def probability_wheel(solutions, weights_list):
    total_score = sum(weights_list)
    probabilities = [x / total_score for x in weights_list] # gets proportion of each solution based on score (adds to 1)
    selection = []
    for i in range(len(solutions)):
        rand = random() # number between 0 and 1
        prob_sum = 0 # cumulative probability
        for idx, prob in enumerate(probabilities):
            prob_sum += prob # adds up to total
            if prob_sum > rand: # cutoff is exceeded
                selection.append(solutions[idx])
                break

    return selection

selected_solutions = probability_wheel(solutions_coords, weights)

In [681]:
def crossover(solutions, cross_rate): # TO DO : IMPLEMENT CROSSOVER RATE
    parents = [(solutions[i], solutions[i + 1]) for i in range(0, len(solutions), 2)]
    cross_solutions = [] # store the newly crossovered solutions
    idxx = len(solutions[0])
    for parent1, parent2 in parents:
        sub_cross_solutions_c1 = {}
        sub_cross_solutions_c2 = {}

        p1_city = [city for city in parent1.keys()]
        p2_city = [city for city in parent2.keys()]
        p1_coords = [coord for coord in parent1.values()]
        p2_coords = [coord for coord in parent2.values()]

        p1_idx_list = [i for i in range(idxx)]
        p2_idx_list = [i for i in range(idxx)]

        shuffle(p1_idx_list)
        shuffle(p2_idx_list)

        start_point_idx = randrange(2, idxx - 3)
        end_point_idx = randrange(start_point_idx + 1, idxx - 2)

        c1_proto_init = [-1 for i in range(idxx)]
        c2_proto_init = [-1 for i in range(idxx)]

        c1_proto = c1_proto_init[:start_point_idx] + p1_idx_list[start_point_idx:end_point_idx] + c1_proto_init[end_point_idx:]
        c2_proto = c2_proto_init[:start_point_idx] + p2_idx_list[start_point_idx:end_point_idx] + c2_proto_init[end_point_idx:]

        for index, element in enumerate(c1_proto):
            if element == -1:
                for i in p2_idx_list:
                    if i not in c1_proto:
                        c1_proto[index] = i

        for index2, element2 in enumerate(c2_proto):
            if element2 == -1:
                for j in p1_idx_list:
                    if j not in c2_proto:
                        c2_proto[index2] = j

        for i in c1_proto:
            sub_cross_solutions_c1[p1_city[i]] = p1_coords[i]
            
        for j in c2_proto:
            sub_cross_solutions_c2[p2_city[j]] = p2_coords[j]
            
        cross_solutions.append(sub_cross_solutions_c1)
        cross_solutions.append(sub_cross_solutions_c2)

    return cross_solutions
        
crossover_results = crossover(selected_solutions, 0.1)

In [682]:
def mutate(solutions, mutate_rate):
    for idx, solution in enumerate(solutions):
        cities = [city for city in solution.keys()]
        coords = [coord for coord in solution.values()]
        for i, current_city in enumerate(cities):
            rand = random()
            rand_idx_2 = randrange(0, len(cities))
            if rand < mutate_rate:
                cities[i], cities[rand_idx_2] = cities[rand_idx_2], cities[i]
                coords[i], coords[rand_idx_2] = coords[rand_idx_2], coords[i]

        solutions[idx] = dict(zip(cities, coords))

    return solutions

mutate_test = mutate(cross_sample, 0.1)

mutate_test[0]

{'Rome': (16, 5),
 'Paris': (8, 12),
 'Madrid': (19, 14),
 'Vienna': (4, 3),
 'Lisbon': (1, 9),
 'Amsterdam': (2, 9),
 'Prague': (1, 12),
 'Stockholm': (10, 19),
 'London': (3, 11),
 'Berlin': (5, 19)}

In [684]:
def genetic_algorithm(cities, max_distance, n_sol, n_cities, n_iterations, cross_rate, mutate_rate, verbose = False):
    coords = determine_coords(cities, max_distance)
    cities_selection = initial_countries(n_sol, n_cities, cities)
    solutions_coords = construct_pop_coords(cities_selection, coords)
    
    optimal_solution = None
    optimal_dist = 10000
    optimal_score = 0
    
    for iteration in range(n_iterations):
        total_distances = calc_total_dist(solutions_coords)
        weights = assign_weights(total_distances)
        selected_solutions = probability_wheel(solutions_coords, weights)

        crossover_results = crossover(selected_solutions, cross_rate)

        mutated_results = mutate(crossover_results, mutate_rate)

        total_distances_after_algo = calc_total_dist(mutated_results)
        weights_new_gen = assign_weights(total_distances_after_algo)

        best_dist = total_distances_after_algo[np.argmax(weights_new_gen)]
        best_sol = mutated_results[np.argmax(weights_new_gen)]

        if verbose:
            print(f"best solution : {best_sol} | total dist : {best_dist}")

        if best_dist < optimal_dist:
            optimal_dist = best_dist
            optimal_solution = best_sol

        solutions_coords = mutated_results

    print(optimal_solution)
    print(optimal_dist)

genetic_algorithm(cities = cities, max_distance = 20, n_sol = 100, n_cities = 10, n_iterations = 1000, cross_rate = 0.4, mutate_rate = 0.2, verbose = False)

{'Vienna': (15, 10), 'Amsterdam': (14, 7), 'Stockholm': (14, 4), 'Lisbon': (8, 4), 'Rome': (6, 7), 'Prague': (1, 10), 'Berlin': (1, 8), 'Paris': (8, 10), 'Madrid': (4, 15), 'London': (5, 13)}
39.518082934690824
